In [39]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
import urllib.request as req
import requests
import time
import re
from urllib.parse import urljoin
import os

In [40]:
each_url =('k1m', 'k2m', 'k3m', 'k1ma', 'kmb')
url = ("https://19ch.tv/km/km.html")

In [41]:
def Subject_Link_Get(self):

    url_x = (f"https://19ch.tv/km/{self}.html")

    response = req.urlopen(url_x)
    parse_html = BeautifulSoup(response, 'html.parser')
    a = parse_html.find_all('a')
    T = parse_html.find('h1').text
    
    return (a, T)

In [43]:
def Lesson_Link_Get(self):
    
    title_list = []
    pdf_list = []
    lesson_list = []

    for i in self:
        if 'href' in i.attrs:
            h = i.attrs['href']

            if re.match(x + '/.*pdf', h):
                temp_url_pdf = urljoin(url, h)
                if not temp_url_pdf in pdf_list:
                    pdf_list.append(temp_url_pdf)
                    
            if re.match(x + '/\w*.html', h):
                temp_url_lesson = urljoin(url, h)
                if not temp_url_lesson in lesson_list:
                        lesson_list.append(temp_url_lesson)
                if i.text != '':
                    title_list.append(i.text)
                    
    
    return (title_list, pdf_list, lesson_list)

In [44]:
def Download_PDF(title, pdf):
    
    for (t, p) in zip(title, pdf):
        req.urlretrieve(p, f'/Users/makidaisuke/Desktop/my_pands/数学/問題集/pdf_dir/{t}.pdf')

In [51]:
def Download_Answer(title, answer):
    
    answer_list = []
    
    for (t, a) in zip(title, answer):
        time.sleep(1)
        r = req.urlopen(a)
        ph = BeautifulSoup(r, 'html.parser')
        answer_link = ph.find_all('a', class_='pure-button pure-button-primary')
        for al in answer_link:
            if al.text == '葉一先生の解答':
                temp_url_answer = urljoin(a, al.attrs['href'])
                answer_list.append(temp_url_answer)
                print(temp_url_answer)
    return answer_list

In [53]:
for x in each_url:
    a, T = Subject_Link_Get(x)
    title, pdf, lesson =  Lesson_Link_Get(a)
    answer = Download_Answer(title, lesson)
    #     Download_PDF(title, pdf)

    df = pd.DataFrame({'Title': title,
                                       'Lesson': lesson,
                                       'Answer': answer,
                                       'Finish': False})
    df.to_csv(f'/Users/makidaisuke/Desktop/my_pands/数学/問題集/{T}.csv')
else:
    print('完了')

https://19ch.tv/km/k1m/ans/k1m1-ans.html
https://19ch.tv/km/k1m/ans/k1m2-ans.html
https://19ch.tv/km/k1m/ans/k1m3-ans.html
https://19ch.tv/km/k1m/ans/k1m4-ans.html
https://19ch.tv/km/k1m/ans/k1m5-ans.html
https://19ch.tv/km/k1m/ans/k1m6-ans.html
https://19ch.tv/km/k1m/ans/k1m7-ans.html
https://19ch.tv/km/k1m/ans/k1m8-ans.html
https://19ch.tv/km/k1m/ans/k1m9-ans.html
https://19ch.tv/km/k1m/ans/k1m10-ans.html
https://19ch.tv/km/k1m/ans/k1m11-ans.html
https://19ch.tv/km/k1m/ans/k1m12-ans.html
https://19ch.tv/km/k1m/ans/k1m13-ans.html
https://19ch.tv/km/k1m/ans/k1m14-ans.html
https://19ch.tv/km/k1m/ans/k1m15-ans.html
https://19ch.tv/km/k1m/ans/k1m16-ans.html
https://19ch.tv/km/k1m/ans/k1m17-ans.html
https://19ch.tv/km/k1m/ans/k1m18-ans.html
https://19ch.tv/km/k1m/ans/k1m19-ans.html
https://19ch.tv/km/k1m/ans/k1m20-ans.html
https://19ch.tv/km/k1m/ans/k1m21-ans.html
https://19ch.tv/km/k1m/ans/k1m22-ans.html
https://19ch.tv/km/k1m/ans/k1m23-ans.html
https://19ch.tv/km/k1m/ans/k1m24-ans.html
h

https://19ch.tv/km/k2m/ans/k2m97-ans.html
https://19ch.tv/km/k2m/ans/k2m98-ans.html
https://19ch.tv/km/k2m/ans/k2m99-ans.html
https://19ch.tv/km/k3m/ans/k3m1-ans.html
https://19ch.tv/km/k3m/ans/k3m2-ans.html
https://19ch.tv/km/k3m/ans/k3m3-ans.html
https://19ch.tv/km/k3m/ans/k3m4-ans.html
https://19ch.tv/km/k3m/ans/k3m5-ans.html
https://19ch.tv/km/k3m/ans/k3m6-ans.html
https://19ch.tv/km/k3m/ans/k3m7-ans.html
https://19ch.tv/km/k3m/ans/k3m8-ans.html
https://19ch.tv/km/k3m/ans/k3m9-ans.html
https://19ch.tv/km/k3m/ans/k3m10-ans.html
https://19ch.tv/km/k3m/ans/k3m11-ans.html
https://19ch.tv/km/k3m/ans/k3m12-ans.html
https://19ch.tv/km/k3m/ans/k3m13-ans.html
https://19ch.tv/km/k3m/ans/k3m14-ans.html
https://19ch.tv/km/k3m/ans/k3m15-ans.html
https://19ch.tv/km/k3m/ans/k3m16-ans.html
https://19ch.tv/km/k3m/ans/k3m17-ans.html
https://19ch.tv/km/k3m/ans/k3m18-ans.html
https://19ch.tv/km/k3m/ans/k3m19-ans.html
https://19ch.tv/km/k3m/ans/k3m20-ans.html
https://19ch.tv/km/k3m/ans/k3m21-ans.html
h

https://19ch.tv/km/k1ma/ans/k1ma18-ans.html
https://19ch.tv/km/k1ma/ans/k1ma19-ans.html
https://19ch.tv/km/k1ma/ans/k1ma20-ans.html
https://19ch.tv/km/k1ma/ans/k1ma21-ans.html
https://19ch.tv/km/k1ma/ans/k1ma22-ans.html
https://19ch.tv/km/k1ma/ans/k1ma23-ans.html
https://19ch.tv/km/k1ma/ans/k1ma24-ans.html
https://19ch.tv/km/k1ma/ans/k1ma25-ans.html
https://19ch.tv/km/k1ma/ans/k1ma26-ans.html
https://19ch.tv/km/k1ma/ans/k1ma27-ans.html
https://19ch.tv/km/k1ma/ans/k1ma28-ans.html
https://19ch.tv/km/k1ma/ans/k1ma29-ans.html
https://19ch.tv/km/k1ma/ans/k1ma30-ans.html
https://19ch.tv/km/k1ma/ans/k1ma31-ans.html
https://19ch.tv/km/k1ma/ans/k1ma32-ans.html
https://19ch.tv/km/k1ma/ans/k1ma33-ans.html
https://19ch.tv/km/k1ma/ans/k1ma34-ans.html
https://19ch.tv/km/k1ma/ans/k1ma35-ans.html
https://19ch.tv/km/k1ma/ans/k1ma36-ans.html
https://19ch.tv/km/k1ma/ans/k1ma37-ans.html
https://19ch.tv/km/k1ma/ans/k1ma38-ans.html
https://19ch.tv/km/k1ma/ans/k1ma39-ans.html
https://19ch.tv/km/k1ma/ans/k1ma

In [11]:
for x in each_url:
    a, T = Subject_Link_Get(x)
    title, pdf, lesson =  Lesson_Link_Get(a)
#     Download_PDF(title, pdf, lesson)